# Mepha - List
## Wichtig
* Manche Adressen enthalten "touring". Nachträglich entfernen
* Spalten evt anpassen

In [190]:
import sys
sys.path.insert(0, '../../../lib/')

import numpy as np
import pandas as pd
import importlib
import pikepdf

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
import tabula

## Unlock PDF
Dieser Teil entfernt den Passwortschutz, damit das PDF in HTML exportiert werden kann

In [191]:
pdf = pikepdf.open('switzerland-2019-disclosure-19.06.2020.pdf')
pdf.save('unlocked.pdf')

In [192]:
columns = [46.8, 183.8, 244.8, 286.5, 372.9, 420.5, 484, 618.5, 657.2, 727.2, 791.2, 858.2, 926.7, 975.8, 1053.2]
df = tabula.read_pdf("unlocked.pdf", pages='all', lattice=False, columns=columns, guess=False, pandas_options={'header': None})

## Format Table

In [193]:
df_export = df.copy()

#Rename
df_export.columns = ['empty0', 'name', 'location', 'country', 'address', 'empty2', 'donations_grants', 'empty4', 'empty5', 'sponsorship', 'registration_fees','travel_accommodation', 'empty6', 'fees', 'related_expenses', 'total']

#Replace Strings
df_export.loc[df_export['name'].str.contains('Health Care Professionals', na=False), 'name'] = np.nan
df_export.loc[df_export['name'].str.contains('Health Care Organisations', na=False), 'name'] = np.nan

df_export['name'] = df_export['name'].fillna('')
df_export['location'] = df_export['location'].fillna('')
df_export['country'] = df_export['country'].fillna('')
df_export['address'] = df_export['address'].fillna('')


#Multiline-Name is on different rows. Put them together
for index, row in df_export.iterrows():
    if (row['country'] == '') and (row['location'] == '') and (row['name'] != ''):
        df_export.iloc[index + 1]['name'] = row['name'] + ' ' + df_export.iloc[index + 1]['name']
        #df_export.iloc[index + 1]['empty2'] = row['empty2']
        
#Multiline-Address is on different rows. Put them together
for index, row in df_export.iterrows():
    if (row['country'] == '') and (row['location'] == '') and (row['address'] != ''):
        df_export.iloc[index + 1]['address'] = row['address'] + ' ' + df_export.iloc[index + 1]['address']
        
#Remove empty rows
df_export.loc[df_export['total'] == '', 'total'] = np.nan
df_export.loc[df_export['name'] == '', 'name'] = np.nan
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.dropna(subset=['name'], how='all')
#df_export = df_export.dropna(subset=['empty2'], how='all')

#Drop Sponsorship = '-'
df_export = df_export[df_export['sponsorship'] != '-']
df_export = df_export[df_export['location'] != '‐']
df_export['name'] = df_export['name'].str.strip()
df_export = df_export[df_export['name'] != 'turing']

#Remove empty columns
df_export.drop(columns=['empty0', 'empty2', 'empty4', 'empty5', 'empty6'], inplace=True)

#Add Type manually
first_hco = 'Swiss Federation of Clinical Neuro‐Societies c/o IMK Institut für Medizin und Kommunikation'
index = df_export[df_export['name'] == first_hco].index[0]
df_export.loc[0: index, 'type'] = 'hcp'
df_export.loc[index:, 'type'] = 'hco'

#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Add Fields
df_export = add_uci(df_export)
df_export = add_plz(df_export)

#Copy PLZ from HCO Location to PLZ
df_export['plz'] = df_export.location.str.extract(r' (\d{4})')
df_export['location'] = df_export.location.str.replace(r' (\d{4})', '')

#Revert name
df_export = revert_name(df_export, ', ')
df_export['name'] = df_export['name'].str.replace("  ", " ")

#basic string conversion
df_export = basic_string_conversion(df_export)
df_export.loc[df_export['name'] == '', 'name'] = np.nan

export_list(df_export, 'mepha')

saved


In [194]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)